In [ ]:
# check the vectors and lambdas
def vectors_R3BP_by_mu(no_crossings, L123, ODE_R3BP, dir,
                                step, t_span, mu, 
                                init_search=100, init_tol=1e-12, 
                                refinement=100, newton_tol = 1e-15,
                                start_cond_tol = 10**-6):
    Li = [compute_Lagrange_pt(mu, L123), 0, 0, 0] 
    Li.extend([1, 0, 0, 0, 
               0, 1, 0, 0, 
               0, 0, 1, 0, 
               0, 0, 0, 1])  # initial conditions and identity matrix
    time_span = 0

    # compute the Jacobian matrix of the RTBP at Li.
    # The eigenvalues of this matrix are the frequencies of the periodic orbit

    A = variational_eq(t_span[0], Li, mu, 1)[4:20].reshape(4,4)
    eigenvalues, eigenvectors = np.linalg.eig(A)
    lambda_pos = eigenvalues[3].real
    lambda_neg = eigenvalues[2].real
    eigvec_pos = eigenvectors[:,3].real
    eigvec_neg = eigenvectors[:,2].real
    if lambda_pos < lambda_neg:
        print("Warning: eigenvalues are not ordered")
        temp = eigvec_pos
        eigvec_pos = eigvec_neg
        eigvec_neg = temp
    if dir == 1:
        if eigvec_pos[1] > 0:
            eigvec_pos = -eigvec_pos
            #eigvec_pos[0] = -eigvec_pos[0]
        v = eigvec_pos
    elif dir == -1:
        v = eigvec_neg
    else:
        raise ValueError("Direction must be 1 or -1")
    init_cond = Li[0:4] + v*start_cond_tol

    
    return lambda_pos, lambda_neg, eigvec_pos, eigvec_neg, init_cond

def starting_conditions_by_mu(mu_list, Li, iregion, dt, t_span, 
                                      refinement, init_tol, refinement_fine,
                                      tol, start_cond_tol):
    lambdas_pos = np.zeros(len(mu_list))
    lambdas_neg = np.zeros(len(mu_list))
    eigvecs_pos = np.zeros((len(mu_list), 4))
    eigvecs_neg = np.zeros((len(mu_list), 4))
    crossing_initials = np.zeros((len(mu_list), 4))
    for j, mu_val in tqdm(enumerate(mu_list, start=0)):
        lambda_pos, lambda_neg, eigvec_pos, eigvec_neg, init_cond = \
            vectors_R3BP_by_mu(1, Li, lambda t, X: ODE_R3BP(t, mu_val, X),
                                iregion, dt, t_span, mu_val, 
                                refinement, init_tol, refinement_fine, tol,
                                start_cond_tol)
        lambdas_pos[j] = lambda_pos
        lambdas_neg[j] = lambda_neg
        eigvecs_pos[j] = eigvec_pos
        eigvecs_neg[j] = eigvec_neg
        crossing_initials[j] = init_cond
        if mu_fail != -1:
            mu_fails.append(mu_val)
    return lambdas_pos, lambdas_neg, eigvecs_pos, \
           eigvecs_neg, crossing_initials

In [ ]:
# test this new function

mu_list = np.arange(0.025, 0.1, 0.0001)
print("number of mu values = ", len(mu_list))

iregion = 1

t0 = 0
tmax = 3
dt = 1
t_span = np.arange(t0, tmax, dt)

# Variables for tweaking the accuracy of the crossing time
refinement = 2000
refinement_fine = 25
init_tol = 1e-7
start_cond_tol = 10**-4

# tol for newton solver:
tol = 1e-6 # even a relatively large tolerance gives good results

# Test the function
lambdas_pos, lambdas_neg, eigvecs_pos, eigvecs_neg, crossing_initials = \
    starting_conditions_by_mu(mu_list, 3, 1, dt, t_span, 
                                      refinement, init_tol, refinement_fine, 
                                      tol, start_cond_tol)

# plot these different variables in different subplots as a function of mu
fig = plt.figure(figsize=(4,3))
ax = fig.add_subplot(111)
ax.scatter(mu_list, lambdas_pos, color='b', label='x-y', s=3)
ax.set_xlabel('mu')
ax.set_ylabel("lambda_pos")
ax.set_title('Unstable manifold of L3')
plt.show()

fig = plt.figure(figsize=(4,3))
ax = fig.add_subplot(111)
ax.scatter(mu_list, lambdas_neg, color='b', label='x-y', s=3)
ax.set_xlabel('mu')
ax.set_ylabel("lambda_neg")
ax.set_title('Unstable manifold of L3')
plt.show()

fig = plt.figure(figsize=(4,3))
ax = fig.add_subplot(111)
ax.scatter(mu_list, eigvecs_pos[:,0], color='b', label='x-y', s=3)
ax.set_xlabel('mu')
ax.set_ylabel("x'")
ax.set_title('Unstable manifold of L3')
plt.show()

fig = plt.figure(figsize=(4,3))
ax = fig.add_subplot(111)
ax.scatter(mu_list, eigvecs_pos[:,1], color='b', label='x-y', s=3)
ax.set_xlabel('mu')
ax.set_ylabel("y'")
ax.set_title('Unstable manifold of L3')
plt.show()

fig = plt.figure(figsize=(4,3))
ax = fig.add_subplot(111)
ax.scatter(mu_list, eigvecs_pos[:,2], color='b', label='x-y', s=3)
ax.set_xlabel('mu')
ax.set_ylabel("vx'")
ax.set_title('Unstable manifold of L3')
plt.show()

fig = plt.figure(figsize=(4,3))
ax = fig.add_subplot(111)
ax.scatter(mu_list, eigvecs_pos[:,3], color='b', label='x-y', s=3)
ax.set_xlabel('mu')
ax.set_ylabel("vy'")
ax.set_title('Unstable manifold of L3')
plt.show()